# Introducción al Procesamiento de Lenguaje Natural

El Procesamiento de Lenguaje Natural Clínico (PLN clínico) se enfoca en analizar y extraer información relevante de textos escritos por profesionales de la salud, como registros médicos e interconsultas. Estos textos contienen valiosa información clínica, pero también presentan desafíos debido a su lenguaje especializado, abreviaturas y variaciones en la escritura. En este curso aprenderás a manejar y procesar textos clínicos en español, utilizando herramientas de análisis automático para apoyar la toma de decisiones en salud.

In [6]:
import datasets # Biblioteca de manejo de conjuntos de datos para procesamiento de lenguaje natural
import pandas as pd # Biblioteca de manejo de conjuntos de datos
import re # Módulo de expresiones regulares
from pathlib import Path # Biblioteca para manejo de paths relativos
import os # Módulo incorporado en Python que proporciona funciones para interactuar con el sistema operativo
import csv # Módulo incorporado en Python que proporciona funciones para leer y escribir archivos CSV

## Importación de datos de texto

El primer paso en el procesamiento de lenguaje natural clínico es la lectura e importación de textos desde archivos, como registros médicos o notas de consulta. Al trabajar con datos de texto, es fundamental manejar correctamente la codificación de caracteres (*encoding*) para asegurar que los textos sean leídos de forma adecuada, especialmente cuando incluyen tildes, eñes o símbolos especiales. Seleccionar el encoding correcto evita errores y problemas de interpretación en los análisis posteriores.

### Codificaciones comunes de texto

- **UTF-8:** El encoding más utilizado y recomendado. Soporta prácticamente todos los caracteres y es ideal cuando no se conoce el origen exacto del archivo.
- **Latin-1 (ISO 8859-1):** Común en archivos creados en entornos Windows o sistemas europeos; soporta muchos idiomas occidentales, incluidos los caracteres latinos.
- **UTF-16:** Utiliza 16 bits por carácter y es útil para textos multilingües o algunos idiomas asiáticos.
- **UTF-32:** Emplea 32 bits por carácter y se usa en aplicaciones muy específicas; rara vez necesario para registros clínicos en español.
- **ASCII:** Solo admite caracteres en inglés sin acentos ni símbolos especiales; no es adecuado para textos en español.

### Leer un archivo de texto con `open()`

Para acceder al contenido de un archivo de texto en Python, se utiliza la función `open()`. Es importante indicar el modo de apertura (`"r"` para lectura) y la codificación de caracteres (`encoding="utf-8"` suele ser la opción adecuada para textos en español).

In [7]:
with open(
    "/workspaces/mt/data/ann_sample/interconsulta.txt",  # Ruta al archivo de texto, modificar según sea necesario
    "r",  # Modo de apertura del archivo: 'r' para lectura
    encoding="utf-8",  # Codificación del archivo, 'utf-8' es común para archivos de texto
) as file:
    txt = file.read()
    
print(txt)

- ANGINA DE PECHO, NO ESPECIFICADA/  - Fundamento Clínico APS: Paciente Hipertensa, con adormecimiento en brazo izquierdo y sensacion de ahogo.


**Nota:**  
Si al ejecutar este código aparecen errores relacionados con la codificación, prueba cambiando `encoding="utf-8"` por `encoding="latin-1"`.  
Asegúrate de que la ruta al archivo sea correcta según la ubicación en tu equipo o entorno de trabajo.

### Leer una tabla de datos con la biblioteca estándar `csv`

Además de archivos de texto simples, es común encontrar datos clínicos organizados en formato de tabla, como archivos CSV (valores separados por comas). Python ofrece la biblioteca estándar `csv` para leer este tipo de archivos de manera eficiente. A continuación se muestra cómo abrir un archivo CSV y leer su contenido fila por fila, convirtiendo cada fila en un diccionario para facilitar el acceso a las columnas por nombre.

In [8]:
# Especifica la ruta al archivo CSV
ruta_csv = "/workspaces/mt/spanish_diagnostics/spanish_diagnostics.csv"  # Modifica según corresponda

# Abrir el archivo CSV y leerlo como una lista de diccionarios
with open(ruta_csv, newline='', encoding='utf-8') as archivo:
    lector = csv.DictReader(archivo)
    for i, fila in enumerate(lector):
        print(fila)  # Cada fila es un diccionario con claves iguales a los nombres de las columnas
        if i == 10:
            break

{'diagnostic': '- ANOMALÍAS DENTOFACIALES (INCLUSO LA MALOCLUSIÓN)\r\n\r\n\r\n DISCREPANCIA DENTOMAXILAR', 'is_dental': '1'}
{'diagnostic': 'OBTRUCCION FOSA NASAL DERECHA', 'is_dental': '0'}
{'diagnostic': 'Perturbación de la actividad y de la atención Trastorno defícit atencional', 'is_dental': '0'}
{'diagnostic': 'M7 PROLAPSO VAGINAL PARED ANTERIOR G11 G 111 ALGIA PELVICA HTA CRONICA', 'is_dental': '0'}
{'diagnostic': 'PIEZA 3 CARIES DENTINARIA PROFUNDA PROXIMA A CAMARA PULPAR, EVALUAR POR ESPECIALIDAD', 'is_dental': '1'}
{'diagnostic': 'pieza n 3.4 tratada endodonticamente, restaurada con ionomero y resina compuesta. Necesita protesis fija por gran pNrdida coronaria', 'is_dental': '1'}
{'diagnostic': 'PZ. 12 TREPANADA', 'is_dental': '1'}
{'diagnostic': 'CARCINOMA TORIODEO', 'is_dental': '0'}
{'diagnostic': 'DISPEPSIA Y METEORISMO', 'is_dental': '0'}
{'diagnostic': 'ASA 1 DENTICION TEMPORAL MORDIDA CRUZADA', 'is_dental': '1'}
{'diagnostic': '- DISMINUCION DE AGUDEZA VISUAL. - ESCATOM

### Leer tablas de datos con DataFrames de `Pandas`

La biblioteca `pandas` es fundamental en ciencia de datos y permite trabajar de forma eficiente con tablas grandes. Para importar un archivo CSV y convertirlo en un *DataFrame*, usamos la función `pd.read_csv()`. Es importante asegurarse de usar la codificación correcta (`encoding`).

In [9]:
# Leer el archivo CSV correctamente con UTF-8 (recomendado)
df = pd.read_csv("/workspaces/mt/spanish_diagnostics/spanish_diagnostics.csv", encoding='utf-8')
df.head()

,diagnostic,is_dental
0,- ANOMALÍAS DENTOFACIALES (INCLUSO LA MALOCLUS...,1
1,OBTRUCCION FOSA NASAL DERECHA,0
2,Perturbación de la actividad y de la atención ...,0
3,M7 PROLAPSO VAGINAL PARED ANTERIOR G11 G 111 A...,0
4,PIEZA 3 CARIES DENTINARIA PROFUNDA PROXIMA A C...,1


Si la codificación no es correcta, los caracteres pueden verse mal o aparecer errores. Por ejemplo, al leer el archivo con `latin-1` podría haber problemas:

In [10]:
# Intentar leer con otra codificación
df_malo = pd.read_csv(ruta_csv, encoding='latin-1')
df_malo.head()

,diagnostic,is_dental
0,- ANOMALÃAS DENTOFACIALES (INCLUSO LA MALOCLU...,1
1,OBTRUCCION FOSA NASAL DERECHA,0
2,PerturbaciÃ³n de la actividad y de la atenciÃ³...,0
3,M7 PROLAPSO VAGINAL PARED ANTERIOR G11 G 111 A...,0
4,PIEZA 3 CARIES DENTINARIA PROFUNDA PROXIMA A C...,1


**Tip:**  
Siempre revisa las primeras filas del DataFrame (`.head()`) para comprobar que los caracteres especiales (tildes, ñ, etc.) se hayan importado correctamente y no ves símbolos extraños o errores.  
Si ves caracteres extraños (`Ã¡`, `Ã±`, etc.), revisa y prueba cambiar el argumento `encoding`.

## 🤗 Datasets

🤗 (HuggingFace) Datasets es una biblioteca de manejo de conjuntos de datos para procesamiento de lenguaje natural que se destaca por la simplicidad de sus métodos y el gran repositorio 🤗 Hub que contiene muchos conjuntos de datos libres para descargar sólo con una linea de Python.

En nuestro curso trabajaremos con `spanish_diagnostics`, un conjunto de datos de nuestro grupo investigación que contiene textos de sospechas diagnósticas de la lista de espera chilena y está etiquetado con el destino de la interconsulta; este destino puede ser `dental` o `no_dental`.

In [11]:
spanish_diagnostics = datasets.load_dataset('fvillena/spanish_diagnostics') # Con esta linea descargamos el conjunto de datos completo

spanish_diagnostics.py:   0%|          | 0.00/1.67k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/3.27M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/70000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/30000 [00:00<?, ? examples/s]

Nuestro conjunto de datos cuenta con 2 particiones, una partición `train` y otra `test`.

In [12]:
spanish_diagnostics

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 70000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 30000
    })
})

En esta clase utilizaremos la partición `train` del conjunto de datos.

In [13]:
spanish_diagnostics["train"]

Dataset({
    features: ['text', 'label'],
    num_rows: 70000
})

Podemos acceder facilmente a atributos de nuestro `Dataset`.

- `shape`: Tal como en muchas otras bibliotecas de python este atributo contiene la forma de nuestro conjunto de datos con la sintaxis `(filas, columnas)`.
- `column_names`: Este atributo contiene el nombre de las características que tiene nuestro conjunto de datos. En nuestro caso tenemos una característica `text`, la cual contiene la hipótesis diagnóstica del conjunto de datos y `label` que contiene el destino al cual fue referido.
- `features`: Este atributo nos describe la clase a la que pertenece cada una de las características. En nuestro caso `text` es un `string` y `label` es del tipo `ClassLabel` con 2 clases con nombre `not_dental` y `dental`.

In [14]:
spanish_diagnostics["train"].shape

(70000, 2)

In [15]:
spanish_diagnostics["train"].column_names

['text', 'label']

In [16]:
spanish_diagnostics["train"].features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_dental', 'dental'], id=None)}

Tal como en muchas otras clases de datos en Python podemos acceder a subconjuntos de datos a través de sus índices.

In [17]:
spanish_diagnostics["train"][0]

{'text': '- ANOMALÍAS DENTOFACIALES (INCLUSO LA MALOCLUSIÓN)\n\n\n DISCREPANCIA DENTOMAXILAR',
 'label': 1}

In [18]:
spanish_diagnostics["train"][:3]

{'text': ['- ANOMALÍAS DENTOFACIALES (INCLUSO LA MALOCLUSIÓN)\n\n\n DISCREPANCIA DENTOMAXILAR',
  'OBTRUCCION FOSA NASAL DERECHA',
  'Perturbación de la actividad y de la atención Trastorno defícit atencional'],
 'label': [1, 0, 0]}

In [19]:
spanish_diagnostics["train"][1,3,5]

{'text': ['OBTRUCCION FOSA NASAL DERECHA',
  'M7 PROLAPSO VAGINAL PARED ANTERIOR G11 G 111 ALGIA PELVICA HTA CRONICA',
  'pieza n 3.4 tratada endodonticamente, restaurada con ionomero y resina compuesta. Necesita protesis fija por gran pNrdida coronaria'],
 'label': [0, 0, 1]}

También podemos acceder a cada una de las características por separado.

In [20]:
spanish_diagnostics["train"]['text'][:3]

['- ANOMALÍAS DENTOFACIALES (INCLUSO LA MALOCLUSIÓN)\n\n\n DISCREPANCIA DENTOMAXILAR',
 'OBTRUCCION FOSA NASAL DERECHA',
 'Perturbación de la actividad y de la atención Trastorno defícit atencional']

Con 🤗 Datasets también podemos trabajar en otras bibliotecas, como por ejemplo importar el conjunto de datos en Pandas.

In [21]:
spanish_diagnostics_train_df = pd.DataFrame(spanish_diagnostics["train"])

In [22]:
spanish_diagnostics_train_df

,text,label
0,- ANOMALÍAS DENTOFACIALES (INCLUSO LA MALOCLUS...,1
1,OBTRUCCION FOSA NASAL DERECHA,0
2,Perturbación de la actividad y de la atención ...,0
3,M7 PROLAPSO VAGINAL PARED ANTERIOR G11 G 111 A...,0
4,PIEZA 3 CARIES DENTINARIA PROFUNDA PROXIMA A C...,1
...,...,...
69995,DM1 Evaluación,1
69996,ABCESO SUBMUCOSO PIEZA 2.6,1
69997,Pbs Inmunodeficiencia,0
69998,"QUISTE SINOVIAL DEL HUECO POPLITEO, DE BAKER",0


Verificamos que nuestro conjunto de datos tiene sus clases balanceadas.

In [23]:
spanish_diagnostics_train_df.label.value_counts()

label
1    35034
0    34966
Name: count, dtype: int64

Si tenemos localmente un conjunto de datos y queremos importarlo a 🤗 Datasets también podemos hacerlo. Aquí importamos el conjunto de datos desde un archivo CSV.

In [24]:
path_datos=str("/workspaces/mt/spanish_diagnostics/spanish_diagnostics.csv")
datasets.load_dataset('csv', data_files=path_datos)

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['diagnostic', 'is_dental'],
        num_rows: 100000
    })
})

## Normalización

Una de las tareas que podemos realizar sobre las características no estructuradas de texto es la normalización. La cual consiste en llevar nuestro texto a una forma más consistente a lo largo del conjunto de datos.

Podemos observar que nuestro conjunto de datos cuenta con una alta inconsistencia respecto al uso de mayúsculas, el uso de tildes y el uso de signos de puntuación.

In [25]:
spanish_diagnostics["train"]["text"][:10]

['- ANOMALÍAS DENTOFACIALES (INCLUSO LA MALOCLUSIÓN)\n\n\n DISCREPANCIA DENTOMAXILAR',
 'OBTRUCCION FOSA NASAL DERECHA',
 'Perturbación de la actividad y de la atención Trastorno defícit atencional',
 'M7 PROLAPSO VAGINAL PARED ANTERIOR G11 G 111 ALGIA PELVICA HTA CRONICA',
 'PIEZA 3 CARIES DENTINARIA PROFUNDA PROXIMA A CAMARA PULPAR, EVALUAR POR ESPECIALIDAD',
 'pieza n 3.4 tratada endodonticamente, restaurada con ionomero y resina compuesta. Necesita protesis fija por gran pNrdida coronaria',
 'PZ. 12 TREPANADA',
 'CARCINOMA TORIODEO',
 'DISPEPSIA Y METEORISMO',
 'ASA 1 DENTICION TEMPORAL MORDIDA CRUZADA']

Para poder llevar todo a minúsculas, simplemente podemos utilizar el método str.lower().

In [26]:
type(spanish_diagnostics["train"]["text"][0])

str

In [27]:
sample_sentence_lower = spanish_diagnostics["train"]["text"][0].lower()
sample_sentence_lower

'- anomalías dentofaciales (incluso la maloclusión)\n\n\n discrepancia dentomaxilar'

### Expresiones regulares

Las expresiones regulares son una herramienta poderosa para manipular y buscar patrones en cadenas de texto. Las expresiones regulares en Python se definen como una secuencia de caracteres que especifican un patrón de búsqueda. Puedes utilizarlas para realizar tareas como validar formatos de cadenas, extraer información específica de un texto o reemplazar partes de una cadena. En python tenemos el paquete re, sus principales funciones son las siguientes:


* re.search(pattern, string): Busca el patrón en toda la cadena y devuelve un objeto "Match" si encuentra una coincidencia. Puedes utilizar métodos como group() para obtener la cadena que coincide con el patrón.

* re.match(pattern, string): Busca el patrón solo al comienzo de la cadena y devuelve un objeto "Match" si encuentra una coincidencia.

* re.findall(pattern, string): Busca todas las coincidencias del patrón en la cadena y devuelve una lista de cadenas que cumplen con el patrón.

* re.sub(pattern, repl, string): Busca todas las coincidencias del patrón en la cadena y las reemplaza con la cadena de reemplazo especificada.

Para definir un patrón de expresión regular, puedes utilizar varios caracteres especiales y secuencias de escape. Algunos de los caracteres especiales comunes incluyen:

* `.` : Coincide con cualquier carácter excepto una nueva línea.
* * Ejemplo: a.b coincide con "aab", "a1b", "a@b", etc., pero no con "a\nb". 
* `*` : Coincide con cero o más repeticiones del elemento anterior.
* * Ejemplo: ab*c coincide con "ac", "abc", "abbc", "abbbc", etc.
* `+` : Coincide con una o más repeticiones del elemento anterior.
* * Ejemplo: ab+c coincide con "abc", "abbc", "abbbc", etc., pero no con "ac".
* `^` : Coincide con el inicio de una cadena o línea.
* * Ejemplo: ^Start coincide con "Start of line", pero no con "End of line: Start".
* `$` : Coincide con el final de una cadena o línea.
* * Ejemplo: end$ coincide con "End of line", pero no con "Start of line: End".
* `?` : Coincide con cero o una repetición del elemento anterior.
* * Ejemplo: colou?r coincide con "color" y "colour".
* `[ ]`: Coincide con cualquier carácter dentro de los corchetes.
* * Ejemplo: `[aeiou]` coincide con cualquier vocal en minúscula.
* `( )` : Agrupación de elementos y captura de grupos.
* * Ejemplo: (ab)+ coincide con "ab", "abab", "ababab", etc.
* `\` : Se utiliza como carácter de escape para caracteres especiales o para dar significado especial a ciertos caracteres.
* * Ejemplo: \d coincide con cualquier dígito, \b coincide con una posición en la cadena donde hay un cambio de caracteres de palabra a no palabra o viceversa.
* `|` : Coincide con uno de los patrones separados por el operador "|".
* * Ejemplo: cat|dog coincide con "cat" o "dog".


Para eliminar todo los caracteres no alfabéticos podemos utilizar un patrón de expresión regular con la sintaxis: `[^a-zñáéíóú]`, la cual se explica como:

- `[^`: Este es un `NO` lógico que invierte todo lo que viene a su derecha.
- `a-z`: Este patrón coincide todos los caracteres de la `a` a la `z` (minúsculas)
- `áéíóú`: Este patrón coincide con todas las vocales con tilde.

Todos estos patrones están concatenados con un `O` lógico.

In [28]:
sample_sentence_lower_alpha = re.sub(r'[^a-zñáéíóú]', ' ', sample_sentence_lower)
sample_sentence_lower_alpha

'  anomalías dentofaciales  incluso la maloclusión     discrepancia dentomaxilar'

Reemplazamos todas las vocales con tilde con con su forma sin tilde.

In [29]:
re.sub('ó', 'o', sample_sentence_lower_alpha)

'  anomalías dentofaciales  incluso la maloclusion     discrepancia dentomaxilar'

Agrupamos todo en una función que normalizará una cadena de texto que le pasemos.

In [30]:
def normalize(text, remove_tildes=True):
    """Normaliza una cadena de texto convirtiéndo todo a minúsculas, quitando los caracteres no alfabéticos y los tildes"""
    text = text.lower()  # Llevamos todo a minúscula
    text = re.sub(
        r"[^A-Za-zñáéíóú]", " ", text
    )  # Reemplazamos los caracteres no alfabéticos por un espacio
    if remove_tildes:
        text = re.sub("á", "a", text)  # Reemplazamos los tildes
        text = re.sub("é", "e", text)
        text = re.sub("í", "i", text)
        text = re.sub("ó", "o", text)
        text = re.sub("ú", "u", text)
    return text

Los objetos del tipo Dataset implementan un método Dataset.map() con el cual podemo aplicar una función a cada una de las instancias de nuesto conjunto de datos. Lo interesante de este método es que aplica la función de manera paralela.

In [31]:
spanish_diagnostics_normalized = spanish_diagnostics["train"].map(
    lambda x: { # Utilizamos una función anónima que devuelve un diccionario
        "normalized_text" : normalize(x["text"]) # Esta es una nueva característica que agregaremos a nuestro conjunto de datos.
    })

Map:   0%|          | 0/70000 [00:00<?, ? examples/s]

Ahora nuestro conjunto de datos cuenta con una nueva característica `normalized_text`.

In [32]:
spanish_diagnostics_normalized[0]

{'text': '- ANOMALÍAS DENTOFACIALES (INCLUSO LA MALOCLUSIÓN)\n\n\n DISCREPANCIA DENTOMAXILAR',
 'label': 1,
 'normalized_text': '  anomalias dentofaciales  incluso la maloclusion     discrepancia dentomaxilar'}

In [33]:
spanish_diagnostics_normalized[0:3]

{'text': ['- ANOMALÍAS DENTOFACIALES (INCLUSO LA MALOCLUSIÓN)\n\n\n DISCREPANCIA DENTOMAXILAR',
  'OBTRUCCION FOSA NASAL DERECHA',
  'Perturbación de la actividad y de la atención Trastorno defícit atencional'],
 'label': [1, 0, 0],
 'normalized_text': ['  anomalias dentofaciales  incluso la maloclusion     discrepancia dentomaxilar',
  'obtruccion fosa nasal derecha',
  'perturbacion de la actividad y de la atencion trastorno deficit atencional']}